This notebook is intended to demonstrate how to extract vessels from a CTA image that contains only brain data (skull has been stripped).

In [1]:
import itk
from itk import TubeTK as ttk

from itkwidgets import view

import numpy as np

In [2]:
InputBaseDir = "E:/UNC-Stroke/UNC/CTP/CTAT-001-CTA/"

CTAFilename = InputBaseDir + "CTA.mha"
CTABrainFilename = InputBaseDir + "CTA-Brain.mha"

imMax = itk.imread(CTAFilename, itk.F)
imBrain = itk.imread(CTABrainFilename, itk.F)

In [3]:
view(imBrain)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [4]:
ImageType = itk.Image[itk.F, 3]

imMath = ttk.ImageMath.New(Input=imBrain)
imMath.Threshold( 0.00001, 4000, 1, 0)
imMath.Erode(10,1,0)
imBrainMaskErode = imMath.GetOutput()
imMath.SetInput(imBrain)
imMath.IntensityWindow(0,300,0,300)
imMath.ReplaceValuesOutsideMaskRange(imBrainMaskErode,0.5,1.5,0)
imBrainErode = imMath.GetOutput()

In [5]:
view(imBrainErode)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [6]:
imMath = ttk.ImageMath[ImageType,ImageType].New()
imMath.SetInput(imBrainErode)
imMath.Blur(1.5)
imBlur = imMath.GetOutput()
imBlurArray = itk.GetArrayViewFromImage(imBlur)

numSeeds = 15
seedCoverage = 20
seedCoord = np.zeros([numSeeds,3])
for i in range(numSeeds):
    seedCoord[i] = np.unravel_index(np.argmax(imBlurArray, axis=None), imBlurArray.shape)
    indx = [int(seedCoord[i][0]),int(seedCoord[i][1]),int(seedCoord[i][2])]
    minX = max(indx[0]-seedCoverage,0)
    maxX = max(indx[0]+seedCoverage,imBlurArray.shape[0])
    minY = max(indx[1]-seedCoverage,0)
    maxY = max(indx[1]+seedCoverage,imBlurArray.shape[1])
    minZ = max(indx[2]-seedCoverage,0)
    maxZ = max(indx[2]+seedCoverage,imBlurArray.shape[2])
    imBlurArray[minX:maxX,minY:maxY,minZ:maxZ]=0
    indx.reverse()
    seedCoord[:][i] = imBrain.TransformIndexToPhysicalPoint(indx)
print(seedCoord)

[[  -1.08078575 -149.83951759   -9.45075989]
 [  -5.04153061 -137.95728302   -7.75329781]
 [ -28.80599976 -145.31295204  -19.63553238]
 [  -2.77824783 -128.90415192  -19.63553238]
 [  -1.64660645 -125.50922775  -32.64940834]
 [ -18.05540657 -130.601614    -23.03045654]
 [ -44.64897919 -179.82801437    4.69475746]
 [ -48.04390335 -145.87877274  -20.20135307]
 [  -9.00227547 -101.74475861  -15.10896683]
 [   1.74831772 -109.10042763  -27.55702209]
 [ -14.66048241 -137.95728302  -38.87343597]
 [ -59.92613792 -148.70787621   -3.22673225]
 [ -53.70211029 -124.94340706  -14.54314613]
 [ -49.17554474 -131.16743469  -28.12284279]
 [ -32.20092392 -117.02191734  -38.30761528]]


In [7]:
# Manually extract a few vessels to form an image-specific training set
vSeg = ttk.SegmentTubes.New(Input=imBrain)
vSeg.SetVerbose(True)
vSeg.SetMinRoundness(0.4)
vSeg.SetMinCurvature(0.002)
vSeg.SetRadiusInObjectSpace( 1 )
for i in range(numSeeds):
    print("**** Processing seed " + str(i) + " : " + str(seedCoord[i]))
    vSeg.ExtractTubeInObjectSpace( seedCoord[i], i )
    
tubeMaskImage = vSeg.GetTubeMaskImage()

**** Processing seed 0 : [  -1.08078575 -149.83951759   -9.45075989]
**** Processing seed 1 : [  -5.04153061 -137.95728302   -7.75329781]
**** Processing seed 2 : [ -28.80599976 -145.31295204  -19.63553238]
**** Processing seed 3 : [  -2.77824783 -128.90415192  -19.63553238]
**** Processing seed 4 : [  -1.64660645 -125.50922775  -32.64940834]
**** Processing seed 5 : [ -18.05540657 -130.601614    -23.03045654]
**** Processing seed 6 : [ -44.64897919 -179.82801437    4.69475746]
**** Processing seed 7 : [ -48.04390335 -145.87877274  -20.20135307]
**** Processing seed 8 : [  -9.00227547 -101.74475861  -15.10896683]
**** Processing seed 9 : [   1.74831772 -109.10042763  -27.55702209]
**** Processing seed 10 : [ -14.66048241 -137.95728302  -38.87343597]
**** Processing seed 11 : [ -59.92613792 -148.70787621   -3.22673225]
**** Processing seed 12 : [ -53.70211029 -124.94340706  -14.54314613]
**** Processing seed 13 : [ -49.17554474 -131.16743469  -28.12284279]
**** Processing seed 14 : [ -3

In [8]:
imMath.SetInput(tubeMaskImage)
imMath.AddImages(imBrain, 200, 1)
blendIm = imMath.GetOutput()
view(blendIm)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [9]:
LabelMapType = itk.Image[itk.UC,3]

trMask = ttk.ComputeTrainingMask[ImageType,LabelMapType].New()
trMask.SetInput( tubeMaskImage )
trMask.SetGap( 4 )
trMask.SetObjectWidth( 1 )
trMask.SetNotObjectWidth( 1 )
trMask.Update()
fgMask = trMask.GetOutput()

In [10]:
view(fgMask)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

In [11]:
enhancer = ttk.EnhanceTubesUsingDiscriminantAnalysis[ImageType,LabelMapType].New()
enhancer.AddInput( imMax )
enhancer.SetLabelMap( fgMask )
enhancer.SetRidgeId( 255 )
enhancer.SetBackgroundId( 128 )
enhancer.SetUnknownId( 0 )
enhancer.SetTrainClassifier(True)
enhancer.SetUseIntensityOnly(True)
enhancer.SetScales([0.43,1.29,3.01])
enhancer.Update()
enhancer.ClassifyImages()

In [12]:
im1vess = itk.SubtractImageFilter( Input1=enhancer.GetClassProbabilityImage(0), Input2=enhancer.GetClassProbabilityImage(1))

imMath.SetInput(imBrain)
imMath.Threshold(0.0001,2000,1,0)
imMath.Erode(2,1,0)
imBrainE = imMath.GetOutput()

imMath.SetInput(im1vess)
imMath.ReplaceValuesOutsideMaskRange(imBrainE, 1, 1, -0.001)
im1vessBrain = imMath.GetOutput()
#view(enhancer.GetClassProbabilityImage(0))
view(im1vessBrain)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [13]:
itk.imwrite( im1vess, InputBaseDir + "CTA-VesselEnhanced.mha", compression=True)

itk.imwrite( im1vessBrain, InputBaseDir + "CTA-Brain-VesselEnhanced.mha", compression=True)